In [49]:
import requests
import json
import os
import dotenv
from datetime import datetime, timedelta
from auth_grant import get_service_token
from mysql_conn import get_mysql_connection
import imaplib
import email
from email.header import decode_header
import re


In [ ]:
IMAP_HOST = 'outlook.office365.com'
EMAIL_USER = 'scheduling@polarisforensics.com'
EMAIL_PASS = os.getenv('SCHEDULING_APP_PASSWORD')  # store securely

mail = imaplib.IMAP4_SSL(IMAP_HOST)
mail.login(EMAIL_USER, EMAIL_PASS)

In [41]:
load_dotenv = dotenv.load_dotenv()
DEPLOYMENT = os.getenv("ACCELO_DEPLOYMENT")
print(f"Deployment: {DEPLOYMENT}")

ACCELO_ACCESS_TOKEN = get_service_token()
print(f"Access Token: {ACCELO_ACCESS_TOKEN}")

Deployment: polarisforensics.api.accelo.com


2025-07-01 14:29:05,120 [INFO] Service token successfully retrieved.


Access Token: jSmAizhH0rwO52RrJPE.Lx_Hq6lcG55zTshoc8JvQpKYlxqppYRMpLVQpeiLiVfH


In [ ]:
ACCELO_BASE_URL = f"https://{DEPLOYMENT}/api/v0/"
headers = {
    "Authorization": f"Bearer {ACCELO_ACCESS_TOKEN}",
    "Accept": "application/json"
}

In [48]:
# ---- Get Fields from DB ----
conn = get_mysql_connection()
cursor = conn.cursor()
cursor.execute("SHOW COLUMNS FROM job")
columns = cursor.fetchall()
fields_str = ",".join(col[0] for col in columns)
cursor.close()
conn.close()

# Now fetch job ID 4331 with all those fields
job_id = 4331
params = {
    "fields": fields_str
}

response = requests.get(f"{ACCELO_BASE_URL}jobs/{job_id}", headers=headers, params=params)
job = response.json().get("response", {})

# Print the results side-by-side for comparison
print(f"✅ Job ID {job_id} from Accelo:\n")
for field in fields_str.split(','):
    print(f"{field:25} : {job.get(field)}")

✅ Job ID 4331 from Accelo:

id                        : 4331
custom_id                 : None
type_id                   : None
manager_id                : None
title                     : Gladiator US Topco, Inc.
engagement_id             : None
engagement_table          : None
affiliation_id            : None
created                   : None
modified                  : None
modified_by               : None
start                     : None
commenced                 : None
due                       : None
completed                 : None
status                    : None
status_id                 : None
paused                    : None
rate_id                   : None
rate_charged              : None
budget_id                 : None
contract_id               : None
client_portal_permissions : None
plan_modified             : None
plan_modified_by          : None
last_interaction          : None
comments                  : None
invoice_template_id       : None
billing_affiliation_id    : 

In [55]:
import csv
import psycopg2
from pg_conn import get_pg_connection

# Path to your CSV file
csv_path = r"C:/Users/Vincent Odine/Downloads/Accelo_Table_Sync_Prioritization__Clean_.csv"
table_name = "accelo_monitor.sync_manifest"

# Connect to PostgreSQL
conn = get_pg_connection()
cur = conn.cursor()

# Open the CSV and insert relevant columns
with open(csv_path, newline='', encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile)
    insert_columns = ["table_name", "sync_priority"]

    for row in reader:
        values = [row[col] for col in insert_columns]
        cur.execute(
            f"""
            INSERT INTO {table_name} ({', '.join(insert_columns)})
            VALUES (%s, %s)
            """,
            values
        )

conn.commit()
cur.close()
conn.close()

print("CSV data inserted successfully.")


CSV data inserted successfully.
